# Retrieval-Augmented OpenAI Agent

In this tutorial, we show you how to use our `RetrieverOpenAI` implementation
to build an agent on top of OpenAI's function API and store/index an arbitrary number of tools. Our indexing/retrieval modules help to remove the complexity of having too many functions to fit in the prompt.

## Initial Setup 

Let's start by importing some simple building blocks.  

The main thing we need is:
1. the OpenAI API (we will use langchain's ChatOpenAI wrapper for convienience here.)
2. a place to keep conversation history 
3. a definition for tools that our agent can use.

In [1]:
from llama_index.agent.utils import FunctionMessage, monkey_patch_langchain

# TODO: right now langchain does not support function messages
#       monkey patch it to support it
monkey_patch_langchain()

/Users/jerryliu/Programming/gpt_index/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import json
from typing import Sequence

from langchain.chat_models import ChatOpenAI
from langchain.memory import ChatMessageHistory
from llama_index.tools import BaseTool, FunctionTool

Let's define some very simple calculator tools for our agent.

In [3]:
def multiply(a: int, b: int) -> int:
    """Multiply two integers and returns the result integer"""
    return a * b

def add(a: int, b: int) -> int:
    """Add two integers and returns the result integer"""
    return a + b

def useless(a: int, b: int) -> int:
    """Toy useless function."""
    pass


multiply_tool = FunctionTool.from_defaults(fn=multiply, name="multiply")
useless_tools = [FunctionTool.from_defaults(fn=useless, name=f"useless_{str(idx)}") for idx in range(28)]
add_tool = FunctionTool.from_defaults(fn=add, name="add")

all_tools = [multiply_tool] + [add_tool] + useless_tools
all_tools_map = {t.metadata.name: t for t in all_tools}

In [4]:
# define an index over these tools
from llama_index import VectorStoreIndex
from llama_index.data_structs.node import Node
import json

# insert into index, define mapping function
index = VectorStoreIndex([])
for idx, (name, tool) in enumerate(all_tools_map.items()):
    print(f'adding tool to index: {name}')
    node_text = json.dumps(tool.metadata.to_openai_function())
    node = Node(node_text, node_info={"tool_name": name})
    index.insert_nodes([node])
    
    
def node_to_tool_fn(node: Node):
    """Return tool given node."""
    openai_fn_dict = json.loads(node.get_text())
    return all_tools_map[openai_fn_dict["name"]]

adding tool to index: multiply
adding tool to index: add
adding tool to index: useless_0
adding tool to index: useless_1
adding tool to index: useless_2
adding tool to index: useless_3
adding tool to index: useless_4
adding tool to index: useless_5
adding tool to index: useless_6
adding tool to index: useless_7
adding tool to index: useless_8
adding tool to index: useless_9
adding tool to index: useless_10
adding tool to index: useless_11
adding tool to index: useless_12
adding tool to index: useless_13
adding tool to index: useless_14
adding tool to index: useless_15
adding tool to index: useless_16
adding tool to index: useless_17
adding tool to index: useless_18
adding tool to index: useless_19
adding tool to index: useless_20
adding tool to index: useless_21
adding tool to index: useless_22
adding tool to index: useless_23
adding tool to index: useless_24
adding tool to index: useless_25
adding tool to index: useless_26
adding tool to index: useless_27


In [5]:
# retriever = index.as_retriever()
# nodes = retriever.retrieve("get multiply fn")
# print(nodes)
# tool = node_to_tool_fn(nodes[0].node)
# print(tool)

## Our `RetrieverOpenAIAgent` Implementation 

We provide a (slightly better) `RetrieverOpenAIAgent` implementation in LlamaIndex, which you can directly use as follows.  

In comparison to the simplified version above:
* it implements the `BaseChatEngine` and `BaseQueryEngine` interface, so you can more seamlessly use it in the LlamaIndex framework. 
* it supports multiple function calls per conversation turn
* it supports async endpoints
* it supports callback and tracing

In [ ]:
# doesn't work with multi-function calls atm
# NOTE: requires a bit of prompt tuning

In [6]:
from llama_index.agent import RetrieverOpenAIAgent

In [17]:
agent = RetrieverOpenAIAgent.from_retriever(
    index.as_retriever(),
    node_to_tool_fn,
    verbose=True
)

In [10]:
agent.chat("What's 212 times 122? Make sure to use Tools")

=== Calling Function ===
Calling function: multiply with args: {
  "a": 212,
  "b": 122
}
Got output: 25864


Response(response='The result of 212 multiplied by 122 is 25,864.', source_nodes=[], extra_info=None)

In [19]:
agent.chat("What's 212 added to 122 ? Make sure to use Tools")

=== Calling Function ===
Calling function: add with args: {
  "a": 212,
  "b": 122
}
Got output: 334


Response(response='The sum of 212 and 122 is 334.', source_nodes=[], extra_info=None)